# Functional MRI Preprocessing Pipeline

In [36]:
# import required packages
import os
import numpy as np
import pandas as pd

In [37]:
# function for viewing full dataframes

def print_full_df(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')


## Set up Directory Structure

In [10]:
# main directory for raw 7 T data (READ ONLY)
raw_data_dir = '/media/forest/wd_1/data_CLMS/0_7T_RAW'

# main directory for preprocessing fmri data
fmri_dir = '/media/forest/wd_1/data_CLMS/functional'

# main directory for preprocessing anatomical
anat_dir = '/media/forest/wd_1/data_CLMS/anatomical'

Create directory to store the raw minc files. 

In [45]:
# create directory to copy over raw minc files

# find all files in raw_data_dir containing string "fmri-FA40"
fp_fmri_minc = !find {raw_data_dir} ~+ -type f -name "*fmri-FA40*"

# create dataframe to store the list of file paths
df_preproc_filepaths = pd.DataFrame({'subject_id':"", 'mnc filepath orig':fp_fmri_minc})

# fill in subject ids from mnc filepaths, where ids are formatted w/regex
df_preproc_filepaths['subject_id'] = df_preproc_filepaths['mnc filepath orig'].str.extract(r'(\w{3}_\d{3}_m\d{2})')

print_full_df(df_preproc_filepaths)

     subject_id                                                                                                mnc filepath orig
0   bbb_052_m00           /media/forest/wd_1/data_CLMS/0_7T_RAW/bbb_052/m00/CLMS-LONDON-7_002-LON-7_bbb_052_m00_fmri-FA40.mnc.gz
1   aab_026_m00           /media/forest/wd_1/data_CLMS/0_7T_RAW/aab_026/m00/CLMS-LONDON-7_002-LON-7_aab_026_m00_fmri-FA40.mnc.gz
2   aac_027_m00           /media/forest/wd_1/data_CLMS/0_7T_RAW/aac_027/m00/CLMS-LONDON-7_002-LON-7_aac_027_m00_fmri-FA40.mnc.gz
3   aac_027_m24           /media/forest/wd_1/data_CLMS/0_7T_RAW/aac_027/m24/CLMS-LONDON-7_002-LON-7_aac_027_m24_fmri-FA40.mnc.gz
4   aad_028_m00           /media/forest/wd_1/data_CLMS/0_7T_RAW/aad_028/m00/CLMS-LONDON-7_002-LON-7_aad_028_m00_fmri-FA40.mnc.gz
5   aad_028_m24           /media/forest/wd_1/data_CLMS/0_7T_RAW/aad_028/m24/CLMS-LONDON-7_002-LON-7_aad_028_m24_fmri-FA40.mnc.gz
6   aae_029_m00           /media/forest/wd_1/data_CLMS/0_7T_RAW/aae_029/m00/CLMS-LONDON-7_002-LON

In [46]:
# manual cleanup: from previous cell output, delete rows 25 and 45 & reset indices
df_preproc_filepaths = df_preproc_filepaths.drop([25, 45])
df_preproc_filepaths = df_preproc_filepaths.reset_index()

print_full_df(df_preproc_filepaths)

    index   subject_id                                                                                                mnc filepath orig
0       0  bbb_052_m00           /media/forest/wd_1/data_CLMS/0_7T_RAW/bbb_052/m00/CLMS-LONDON-7_002-LON-7_bbb_052_m00_fmri-FA40.mnc.gz
1       1  aab_026_m00           /media/forest/wd_1/data_CLMS/0_7T_RAW/aab_026/m00/CLMS-LONDON-7_002-LON-7_aab_026_m00_fmri-FA40.mnc.gz
2       2  aac_027_m00           /media/forest/wd_1/data_CLMS/0_7T_RAW/aac_027/m00/CLMS-LONDON-7_002-LON-7_aac_027_m00_fmri-FA40.mnc.gz
3       3  aac_027_m24           /media/forest/wd_1/data_CLMS/0_7T_RAW/aac_027/m24/CLMS-LONDON-7_002-LON-7_aac_027_m24_fmri-FA40.mnc.gz
4       4  aad_028_m00           /media/forest/wd_1/data_CLMS/0_7T_RAW/aad_028/m00/CLMS-LONDON-7_002-LON-7_aad_028_m00_fmri-FA40.mnc.gz
5       5  aad_028_m24           /media/forest/wd_1/data_CLMS/0_7T_RAW/aad_028/m24/CLMS-LONDON-7_002-LON-7_aad_028_m24_fmri-FA40.mnc.gz
6       6  aae_029_m00           /media/forest/w

In [ ]:
# create working directory if applicable
working_dir_name = '0.1_raw_minc'

fmri_dir  =  


In [47]:
# copy the selected minc files to working directory
for subject, filepath in zip(df_preproc_filepaths['subject_id'], df_preproc_filepaths['mnc filepath orig']):

    !echo "Subject: {subject} 
    # if 

bbb_052_m00
aab_026_m00
aac_027_m00
aac_027_m24
aad_028_m00
aad_028_m24
aae_029_m00
aaf_030_m00
aaf_030_m24
aah_032_m00
aah_032_m24
aai_033_m00
aai_033_m24
aaj_034_m00
aaj_034_m24
aal_036_m00
aal_036_m24
aam_037_m00
aao_039_m00
aao_039_m24
aar_042_m00
aas_043_m00
aaw_047_m00
aay_049_m00
abc_001_m24
bbc_053_m00
bbd_054_m00
bbe_055_m00
bbf_056_m00
bbi_059_m00
bbk_061_m00
bbl_062_m00
bcd_002_m24
cde_003_m24
def_004_m00
def_004_m24
efg_005_m00
efg_005_m24
fgh_006_m00
ghi_007_m00
ghi_007_m24
hij_008_m00
ijk_009_m24
klm_011_m24
lmn_012_m00
lmn_012_m24
mno_013_m00
opq_015_m00
opq_015_m24
pqr_016_m00
pqr_016_m24
rst_018_m00
rst_018_m24
stu_019_m00
stu_019_m24
tuv_020_m00
tuv_020_m24
uvw_021_m00
uvw_021_m24
vwx_022_m00
wxy_023_m00
xyz_024_m00
xyz_024_m24


Convert minc files to nifti for preprocessing. 

In [ ]:
# create new directory for nifti files
newdir_name = '0.2_raw_nifti'

## Part 1: Prepare Functional Data

via: https://en.wikibooks.org/wiki/Neuroimaging_Data_Processing/Processing/Steps/Slice_Timing

- first steps: 
    - realignment
    - motion correction
    - slice timing correction via temporal derivatives
- notes: 
    - data characteristics: short TR, interleaved slice order, potentially severe head motion among SPMS group
    - slice acquisition order was Siemens interleaved
    - slice timing correction method chosen based on above characteristics of our data

In [ ]:
# 1.1 realignment to middle volume (#239)



In [ ]:
# 1.2 motion correction & identifying outliers



In [ ]:
# 1.3 slice timing correction via temporal derivatives
 

## Part 2: Prepare Anatomical Data
- part 1: brain extraction via FSL BET
- part 2: registration of high-res anatomical data to template
- transformation matrix is then used on functional data for final registration step

In [ ]:
# preprocessing step: 

In [ ]:
# preprocessing step: 

In [ ]:
# preprocessing step: 

## Part 3: Registration & Normalization
- registration of fmri to high resolution t1w mp2rage anatomical data via boundary-based registration
- followed by normalization to template (MNI152 2mm)

## Part 4: Smoothing